<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_ODEs_(Ordinary_Differential_Equations).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchdiffeq

In [ ]:
import torch
from torch import nn
from torchdiffeq import odeint

# Define the dynamics of the ODE
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        # Introduce a non-linear dynamics layer
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.ReLU(),
            nn.Linear(50, 2)
        )

    def forward(self, t, x):
        return self.net(x)

# ODE Block
class ODEBlock(nn.Module):
    def __init__(self, ode_func):
        super(ODEBlock, self).__init__()
        self.ode_func = ode_func

    def forward(self, x):
        # Solve ODE from t=0 to t=1
        t = torch.linspace(0, 1, steps=10)  # Time interval
        out = odeint(self.ode_func, x, t)
        return out[-1]  # Return the state at t=1

# Model that uses the ODE block
class ODEModel(nn.Module):
    def __init__(self):
        super(ODEModel, self).__init__()
        self.ode_block = ODEBlock(ODEFunc())  # ODE solver block
        self.classifier = nn.Sequential(
            nn.Linear(2, 50),
            nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        x = self.ode_block(x)  # Pass input through ODE block
        return self.classifier(x)  # Output prediction

# Generate toy dataset for training
def create_toy_data():
    x = torch.rand(100, 2)  # 100 samples, 2 features each
    y = (x.sum(dim=1) > 1).float().unsqueeze(1)  # Simple binary labels
    return x, y

# Training loop
def train_model(model, data, target, epochs=100, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()  # Binary classification loss

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        loss = loss_fn(output, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print progress
        if epoch % 10 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

# Example usage
model = ODEModel()
data, target = create_toy_data()
train_model(model, data, target, epochs=100, lr=0.01)

# Test the model
with torch.no_grad():
    predictions = torch.sigmoid(model(data)) > 0.5
    accuracy = (predictions == target).float().mean().item()
    print(f"Training Accuracy: {accuracy:.2f}")